In [ ]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

In [ ]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

# Paths and Names

In [ ]:
### paths and names

PROJECT_NAME = 'news-topic-keyphrase-generation-model-dev'
RUN_ID = 'v2'

DATA_PATH = 'data/preprocess_v2.pickle'

MODEL_CHECKPOINT = 'ainize/gpt-j-6B-float16'
model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()

METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, '.log')

run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

print(run_name)
print(output_dir)

!mkdir -p {SAVE_PATH}

In [ ]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

# Training Args

In [ ]:
report_to="wandb"

num_train_epochs = 15
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=128
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [ ]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [ ]:
prefix = "generate keyphrases: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

- 모델의 예측값과 위에서 전처리한 라벨을 활용하여 평가지표를 출력하는 함수를 선언합니다.
- ROUGE를 평가지표로 사용합니다.
    - ROUGE-N (N-gram) scoring
    - ROUGE-L (Longest Common Subsequence) scoring
        - sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
        - summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

- csv로 저장된 데이터를 불러와 9:1로 훈련과 검증에 사용합니다.
- 선언한 전처리 함수를 적용하여 데이터세트를 생성합니다.

In [ ]:
data_df = pd.read_pickle(DATA_PATH)

In [ ]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2, seed=100)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [ ]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

In [ ]:
tokenizer.decode(train_dataset['input_ids'][0])

In [ ]:
tokenizer.decode(train_dataset['labels'][0])

# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, model=model, padding=True, mlm=False)

trainer = Trainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

- 훈련을 시작합니다.

In [ ]:
trainer.train()

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [ ]:
# keep = [
#     'added_tokens.json',
#     'config.json',
#     'pytorch_model.bin',
#     'special_tokens_map.json',
#     'tokenizer.json',
#     'tokenizer_config.json',
#     'vocab.txt'
# ]

# ckpts = os.listdir(output_dir)
# for ckpt in ckpts:
#     ckpt = os.path.join(output_dir, ckpt)
#     for item in os.listdir(ckpt):
#         if item not in keep:
#             os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [ ]:
preds = trainer.predict(eval_dataset)

In [ ]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')